# Output Parsers - Formatando saídas

Como retornar dados estruturados de um modelo?

É frequentemente útil que um modelo retorne uma saída que corresponda a um esquema específico. Um caso de uso comum é a extração de dados de um texto para inseri-los em um banco de dados ou utilizá-los em algum outro sistema subsequente. Nesta aula abordaremos algumas estratégias para obter saídas estruturadas de um modelo.

## Estruturando saídas de chat - StrOutputParser

O formatador mais simples do LangChain é o StrOutputParser. Ele é utilizado para convertermos saídas do modelo no formato de conversão para formato texto. É uma atividade bem comum, levando em consideração que maior parte das LLMs que utlizamos com LangChain são acessadas através dos ChatModels.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente engraçado e se chama {nome_assistente}"),
    ("human", "{pergunta}"),
])

chat_template.format_messages(nome_assistente="Asimov", pergunta="Qual é o seu nome?")

In [ ]:
prompt = chat_template.invoke({
  'nome_assistente': 'Asimov',
  'pergunta': 'Qual é o seu nome?'
})
prompt

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

resposta = chat.invoke(prompt)
resposta

## StrOutputParser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

output_parser.invoke(resposta)

## Dando um spoiler de chains

In [ ]:
# output de chat_template passa para chat, que passa para output_parser.
chain = chat_template | chat | output_parser

chain.invoke({
  'nome_assistente': 'Asimov',
  'pergunta': 'Qual é o seu nome?'
})

## Estruturando saídas mais complexas - Pydantic

### Utilizando .with_stractured_output()

Esta é a maneira mais fácil e confiável de obter saídas estruturadas. O método with_structured_output() é implementado para modelos que fornecem APIs nativas para estruturar saídas, como chamadas de ferramentas/funções ou modo JSON, e aproveita essa capacidades internamente.

Este modelo recebe um esquema como entrada, que específica os nomes, tipos e descrições dos atributos desejados na saída. Ele retorna um objeto similar a um Runnable, exceto que, em vez de gerar strings ou mensagens, produz objetos correspondentes ao esquema fornecido. O esquema pode ser especificado como uma classe TypedDict, um JSON Schema ou uma classe Pydantic.

In [ ]:
# Pydantic é usado para validação de dados
from pydantic import BaseModel, Field
from typing import Optional

class Piada(BaseModel):
  """Piada para contar aos usuários"""
  introducao: str = Field(description="A introdução da piada")
  piada: str = Field(description="A conclusão da piada")
  avaliacao: Optional[int] = Field(description="O quão engraçado é a piada de 0 a 10?")

llm_estruturada = chat.with_structured_output(Piada)
resposta = llm_estruturada.invoke("Conte uma piada sobre programadores")
resposta

In [ ]:
print(f"Introdução: {resposta.introducao}")
print(f"Piada: {resposta.piada}")
print(f"Avaliação: {resposta.avaliacao}")

## Um exemplo mais prático

Digamos que temos a seguinte review de um produto:

> "Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."

E eu quero que o modelo de linguagem processe esta review para estruturá-la no seguinte formato:

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado"]
}
```

In [ ]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem 
quatro configurações: sopro de vela, brisa suave, cidade ventosa 
e tornado. Chegou em dois dias, bem a tempo para o presente de 
aniversário da minha esposa. Acho que minha esposa gostou tanto 
que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho 
usado em todas as manhãs alternadas para limpar as folhas do 
nosso gramado. É um pouco mais caro do que os outros sopradores 
de folhas disponíveis no mercado, mas acho que vale a pena pelas 
características extras."""

In [ ]:
from pydantic import BaseModel, Field

class AvaliacaoReview(BaseModel):
    """Avalia review do cliente"""
    presente: bool = Field(description='Verdadeiro se foi para presente e False se não foi')
    dias_entrega: int = Field(description='Quantos dias para entrega do produto')
    percepcao_valor: list[str] = Field(description='Extraia qualquer frase sobre o valor ou \
    preço do produto. Retorne uma lista.')

llm_estruturada = chat.with_structured_output(AvaliacaoReview)
resposta = llm_estruturada.invoke(review_cliente)
resposta

In [ ]:
from pydantic import BaseModel, Field

class Avaliacao(BaseModel):
    """Avaliação do produto com base no review do cliente"""
    descricao_produto: str = Field(description="Breve descrição do produto")
    entrega: bool = Field(description="Cliente ficou satisfeito com a entrega")
    produto: bool = Field(description="Cliente ficou satisfeito com o produto")
    atendimento: bool = Field(description="Cliente ficou satisfeito com o atendimento")
    satisfacao: int = Field(description="Satisfação geral do cliente com a compra")

llm_estruturada = chat.with_structured_output(Avaliacao)
resposta = llm_estruturada.invoke(review_cliente)
resposta

In [ ]:
print(f"Descrição: {resposta.descricao_produto}")
print(f"Entrega: {resposta.entrega}")
print(f"Produto: {resposta.produto}")
print(f"Atendimento: {resposta.atendimento}")
print(f"Satisfação: {resposta.satisfacao}")